In [28]:
#!pip install plotly
#!pip install folium

# Auswertung von Daten der deutschen Bahn

## Einleitung

<img src="docs/schienen.png" alt="schienen" width="750" />

[**Schienennetz vor dem Kollaps**](https://www.tagesschau.de/wirtschaft/unternehmen/bahn-chaos-101.html) - so titeln Marie Blöcher, Nils Naber und Isabel Schneider vom NDR. Die Deutsche Bahn habe zwar ehrgeizige Ziele, allerdings ist Jahrelang zu wenig Geld ins Netz geflossen.<br>
Rund 60 Milliarden Euro müssten laut DB ausgegeben werden, um alle Probleme im Netz zu beheben, die sich über die vergangenen Jahre angesammelt haben. Der Zustand von Strecken und Gleisen wurde über viele Jahre vernachlässigt, sagt Bahnexperte Christian Böttger.

Die Bahn steht aktuell in keinem guten Licht. Zu viele Verspätungen, Zugausfälle und marode Infrastruktur.<br>
**Doch wie steht es wirklich um den Zustand der Bahn?**

In dieser Analyse wird auf Daten der Deutschen Bahn zugegriffen, um dieser Frage auf den Grund zu gehen.<br>
Die Bahn stellt über den `API Marketplace` eine Fülle an Daten offen und kostenfrei zur Verfügung.

<img src="docs/bahn_apis.png" alt="schienen" width="750" />

Im Rahmen dieses Projekts betrachen wir folgende Daten:
- Als Grundlage dienst RIS::Stations, darüber lassen sich alle deutschen Bahnhöfe abrufen.
- FaSta - Station Facility Status gibt Auskunft über den Zustand der Bahnhöfe
- Railway-Stations Pictures ermöglich den Zugriff auf Bilder jedes einzelnen Bahnhofs.
-  ...

## Daten abrufen

Über folgende APIs werden die Daten im JSON-Format abgerufen, in pandas dataframes umgewandelt und gespeichert.

```
url_parking = 'https://apis.deutschebahn.com/db-api-marketplace/apis/parking-information/db-bahnpark/v2/'
url_ris_stations = 'https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/'
url_rw_stations = 'https://apis.deutschebahn.com/db-api-marketplace/apis/api.railway-stations.org/photoStationById/'
```

Häufig könne nicht alle Daten auf einmal abgerufen werden, daher müssen mehrere Aufrufe gemacht werden.
Anschlißend ist ein mapping der (json) DB-Datenstruktur auf einen (flaches) Data Frame nötig.

Da der Abruf der Daten einige Minuten dauert, ist dieser und die eigentliche Auswertung getrennt in zwei verschiedenen files.

## Imports

Wir nutzen Dateien im Pickel-Format, um ganze Dataframes zwischen dem Scraping-Process und der Datenauswertung auszutauschen.<br>
Für die Visualisierung wird Plotly und Folium genutzt.

In [29]:
import pandas as pd
import numpy as np

import pickle
import plotly.express as px
import folium

from geopy.geocoders import Nominatim

## Daten laden

Hier werden die Daten aus den Pickel-Files in Dataframes geladen. Der Vorteil dabei ist, dass die Struktur und die Datenformate dabei beibehalten werden.<br>
Das Data Cleaning wird allerdings hier umgesetzt.

In [30]:
pkl_file = open('stations.pkl', 'rb')
df_stations = pickle.load(pkl_file)
pkl_file.close()

In [31]:
pkl_file = open('stopplaces.pkl', 'rb')
df_stopplaces = pickle.load(pkl_file)
pkl_file.close()

Da es mehrere Bilder von einer Haltestelle geben kann, wir allerdings nur eins brauchen, wird hier gefiltert.

In [32]:
pkl_file = open('station_images.pkl', 'rb')
df_station_images = pickle.load(pkl_file)
pkl_file.close()

df_images = pd.DataFrame.from_dict({k: v for k, v in df_station_images.items() if v and len(v) == 1}).T
df_images.columns = ['image']
df_images = df_images.reset_index()

Wir sehen, dass die verschiedenen Daten, die eigentlich zusammengehören, von der Anzahl her nicht komplett zusammenpassen.<br>
Vorallem Haltestellen gibt es einige weniger.

In [33]:
print(f'Stations: {df_stations.shape}')
print(f'Station images: {df_images.shape}')
print(f'Stopplaces: {df_stopplaces.shape}')

Stations: (5690, 16)
Station images: (5627, 2)
Stopplaces: (4208, 9)


### Bahnhöfe

Das Data Frame der Bahnhöfe enthält alle Haltestellen der Deutschen Bahn in Deutschland.<br>
Jeder Bahnhof hat eine eindeutige `id`. Zusältzich wird beispielsweise der Name, die Adresse und Geo-Koordinaten mitgeliefert.

In [34]:
df_stations.head(3)

,id,name,metropolis,street,houseNumber,postalCode,city,state,country,stationCategory,owner,organisationalUnit,countryCode,latitude,longitude,timeZone
0,1,Aachen Hbf,{},Bahnhofstr.,2a,52064,Aachen,Nordrhein-Westfalen,DE,CATEGORY_2,DB S&S,RB West,DE,50.767800,6.091499,Europe/Berlin
1,1000,Burkhardswalde-Maxen,{},Gesundbrunnen,60c,01809,Müglitztal-Burkhardswalde,Sachsen,DE,CATEGORY_7,DB S&S,RB Südost,DE,50.925146,13.838369,Europe/Berlin
2,1001,Burkhardtsdorf,{},Bahnhofstraße,NaN,09235,Burkhardtsdorf,Sachsen,DE,CATEGORY_6,DB Regio-Netze,Erzgebirgsbahn (EGB),DE,NaN,NaN,Europe/Berlin


#### Bilder von Bahnhöfen

Nutzer können Bilder zu Bahnhöfen hochladen.
Das Data Frame enthält Links zu diesen Bildern.
Die Spalte `index` referenziert die Spalte `id` der Bahnhöfe.

In [35]:
df_images.head(3)

,index,image
0,1,https://api.railway-stations.org/photos/de/1_1...
1,1000,https://api.railway-stations.org/photos/de/100...
2,1001,https://api.railway-stations.org/photos/de/100...


#### Haltestellen

Haltestellen enthalten viele Informationen der Bahnhöfe, zusätzlich aber auch die Transportmittel und welcher Verkehrsbund hier fährt.

In [36]:
df_stopplaces.head(3)

,id,name,availableTransports,transportAssociations,countryCode,state,timeZone,latitude,longitude
0,1,Aachen Hbf,"[INTERCITY_TRAIN, CITY_TRAIN, BUS, REGIONAL_TR...","[AAV, VRS]",DE,NW,Europe/Berlin,50.767800,6.091499
1,1000,Burkhardswalde-Maxen,[REGIONAL_TRAIN],[VVO],DE,SN,Europe/Berlin,50.925146,13.838369
2,1001,Burkhardtsdorf,[REGIONAL_TRAIN],[VMS],DE,SN,Europe/Berlin,50.733196,12.932137


## Explorative Datenanalyse

### Datentypen

Als erstest prüfen wir, ob die Data Frames korrekte Datentypen haben und korrigieren sie entsprechend.<br>
Die Spalte `id` soll immer vom Typ *int* sein, um sie später besser zusammenführen zu können.

In [37]:
df_stations['id'] = df_stations['id'].astype(int)
df_stations.dtypes

id                      int32
name                   object
metropolis             object
street                 object
houseNumber            object
postalCode             object
city                   object
state                  object
country                object
stationCategory        object
owner                  object
organisationalUnit     object
countryCode            object
latitude              float64
longitude             float64
timeZone               object
dtype: object

In [38]:
df_stopplaces['id'] = df_stopplaces['id'].astype(int)
df_stopplaces.dtypes

id                         int32
name                      object
availableTransports       object
transportAssociations     object
countryCode               object
state                     object
timeZone                  object
latitude                 float64
longitude                float64
dtype: object

In [39]:
df_images['index'] = df_images['index'].astype(int)
df_images.dtypes

index     int32
image    object
dtype: object

### Fehlende Werte

Als nächsten wird auf fehlende Werte geprüft, um zu schauen, ob hier etwas zu tun ist.

In [40]:
df_stations.isna().sum()

id                      0
name                    0
metropolis              0
street                  8
houseNumber           893
postalCode              7
city                    4
state                   0
country                 0
stationCategory        12
owner                   0
organisationalUnit      0
countryCode             0
latitude              282
longitude             282
timeZone                0
dtype: int64

Das Hausnummernfeld fehlt sehr oft, da wir diese Information aber hier nicht brauchen, ist das kein Problem.

Die Angaben für Latitude und Longitude fehlen auch häufig, hier kann über die Adresse versucht werden, die Werte herauszufinden.<br>
Da das Nachschauen der Werte einige Zeit in Anspruch nimmt, ist dieser Code auskommentiert.<br>

Grundsätzlich wird aber anhand der Spalten `postalCode`, `city`, `state` und `country` mithilfe des Pakets aus der Vorlesung `geopy` versucht, die Geo-Koordinaten aufzulösen.

In [41]:
# geolocator = Nominatim(user_agent="my_app")

# filtered_rows = df_stations[df_stations['latitude'].isnull()]
# result = {}
# # Print the entire row for each entry with NaN latitude
# for index, row in filtered_rows.iterrows():
#     try:
#         address = f'{row["postalCode"]} {row["city"]} {row["state"]} {row["country"]}'
#         result[row['id']] = geolocator.geocode(address)
#     except:
#         pass

In [42]:
# dict={}
# # extract lat/lon
# for index, entry in result.items():
#     if entry:
#         dict[index] = { 'latitude': entry[1][0], 'longitude': entry[1][1] }

# geocode_result = pd.DataFrame().from_dict(dict).T
# geocode_result['id'] = geocode_result.index
# geocode_result['id'] = geocode_result['id'].astype(int)

Die Ergebnisse landen wieder in einem Pickel-File

In [43]:
# output = open('manual_geocode_results.pkl', 'wb')
# pickle.dump(geocode_result, output)
# output.close()

In [44]:
pkl_file = open('manual_geocode_results.pkl', 'rb')
geocode_result = pickle.load(pkl_file)
pkl_file.close()

In [45]:
df_stations

,id,name,metropolis,street,houseNumber,postalCode,city,state,country,stationCategory,owner,organisationalUnit,countryCode,latitude,longitude,timeZone
0,1,Aachen Hbf,{},Bahnhofstr.,2a,52064,Aachen,Nordrhein-Westfalen,DE,CATEGORY_2,DB S&S,RB West,DE,50.767800,6.091499,Europe/Berlin
1,1000,Burkhardswalde-Maxen,{},Gesundbrunnen,60c,01809,Müglitztal-Burkhardswalde,Sachsen,DE,CATEGORY_7,DB S&S,RB Südost,DE,50.925146,13.838369,Europe/Berlin
2,1001,Burkhardtsdorf,{},Bahnhofstraße,NaN,09235,Burkhardtsdorf,Sachsen,DE,CATEGORY_6,DB Regio-Netze,Erzgebirgsbahn (EGB),DE,NaN,NaN,Europe/Berlin
3,1002,Bürstadt,{},Bahnhofsallee,17,68642,Bürstadt,Hessen,DE,CATEGORY_6,DB S&S,RB Mitte,DE,49.645769,8.458188,Europe/Berlin
4,1005,Buschow,{},Bahnhofstr.,28,14715,Märkisch Luch OT Buschow,Brandenburg,DE,CATEGORY_6,DB S&S,RB Ost,DE,52.592203,12.628996,Europe/Berlin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5685,995,Burgstädt,{},Bahnhofstr.,1,09217,Burgstädt,Sachsen,DE,CATEGORY_6,DB S&S,RB Südost,DE,50.915817,12.812707,Europe/Berlin
5686,996,Burgstall (Murr),{},Bahnhofstr.,1,71576,Burgstetten,Baden-Württemberg,DE,CATEGORY_6,DB S&S,RB Südwest,DE,48.928647,9.369932,Europe/Berlin
5687,997,Steinfurt-Burgsteinfurt,{},Bahnhofsplatz,6,48565,Steinfurt-Burgsteinfurt,Nordrhein-Westfalen,DE,CATEGORY_6,DB S&S,RB West,DE,52.147384,7.329340,Europe/Berlin
5688,998,Burgthann,{},Bahnhofstr.,40,90559,Burgthann,Bayern,DE,CATEGORY_5,DB S&S,RB Süd,DE,49.342474,11.309307,Europe/Berlin


Nun liegen die vorhanden Geo-Koordinaten im Data Frame `df_stations` und die neu ermittelten in `geocode_result`.<br>
Um nun das Ergebnis aus beiden Tabellen zu bekommen, wird die Funktion `combine_first` genutzt.

In [46]:
df_stations = df_stations.set_index('id').combine_first(geocode_result.set_index('id')).reset_index()

Anstelle von **282** fehlenden Werten sind es jetzt nur noch **24**!<br>
Die restlichen werden aufgrund der geringen Anzahl ignoriert.

In [47]:
df_stations.isna().sum()

id                      0
city                    4
country                 0
countryCode             0
houseNumber           893
latitude               24
longitude              24
metropolis              0
name                    0
organisationalUnit      0
owner                   0
postalCode              7
state                   0
stationCategory        12
street                  8
timeZone                0
dtype: int64

In [48]:
df_stopplaces.isna().sum()

id                       0
name                     0
availableTransports      0
transportAssociations    0
countryCode              0
state                    6
timeZone                 0
latitude                 0
longitude                0
dtype: int64

Die Haltestellen scheinen eine bessere Datenqualität zu haben, hier gibt es keine Probleme, die betrachtet werden müssen.

### Bringen wir die Daten zusammen!

Jetzt können alle Daten zusammengebracht werden.

Dazu werden zunächst alle doppelten Spalten entfernt und dann die Tabellen mithilfe zweier `join` zusammengefügt.<br>
Da der Data Frame für die Bilder einen anderen Schlüssel hat wie `df_stations`, muss hier `pd.merge` verwendet werden.

In [49]:
df_stopplaces.drop(columns=['name', 'state', 'countryCode', 'latitude', 'longitude','timeZone'], inplace=True)
df = df_stations.join(df_stopplaces, on='id', how='left', rsuffix='_s')
df = pd.merge(left=df, right=df_images, left_on=['id'], right_on=['index'], how='left')
df.drop(columns=['id_s','index','country'], inplace=True)

In [50]:
df.isna().sum()

id                          0
city                        4
countryCode                 0
houseNumber               893
latitude                   24
longitude                  24
metropolis                  0
name                        0
organisationalUnit          0
owner                       0
postalCode                  7
state                       0
stationCategory            12
street                      8
timeZone                    0
availableTransports      2516
transportAssociations    2516
image                      63
dtype: int64

Nach dem join wird deutlich, was am Anfang bei den `shapes` bereits zu sehen war. Wir haben weniger Datensätze in den Haltestellen als bei den Bahnhöfen.

Auf der anderen Seite fehlen nur 63 Bilder, das ist gut.

### Einfache Datenvisualisierung

Schauen wir uns an, was wir an Daten haben und welche Fragen sich daraus ergeben:

- Wir können überprüfen, wie die `organisationalUnit` mit dem `owner` korrelieren, vielleicht auch mit `transportAssociations`.
- Wir können überprüfen, ob wir tatsächlich nur deutsche Stationen haben.
- Wir können die verfügbaren `transportAssociations` überprüfen.
- Natürlich können wir uns die Geo-Koordinaten anschauen.
- Es ist noch fraglich, wofür die Werte von `stationCategory` stehen.

In [51]:
df.head(1)

,id,city,countryCode,houseNumber,latitude,longitude,metropolis,name,organisationalUnit,owner,postalCode,state,stationCategory,street,timeZone,availableTransports,transportAssociations,image
0,1,Aachen,DE,2a,50.7678,6.091499,{},Aachen Hbf,RB West,DB S&S,52064,Nordrhein-Westfalen,CATEGORY_2,Bahnhofstr.,Europe/Berlin,[REGIONAL_TRAIN],[VVO],https://api.railway-stations.org/photos/de/1_1...


In [52]:
def plot_counts(column):
    counts = df[column].value_counts().reset_index()
    counts.columns = [column, 'count']

    fig = px.bar(counts, x=column, y='count', barmode='group', text='count')
    fig.show()

Die meisten Bahnhöfe gehören der DB Station&Service AG. Laut ihrer [Webseite](https://www.deutschebahn.com/de/konzern/konzernprofil/Konzernunternehmen/db_station_service_ag-6879530), unterhalten sie rund `5.400` Bahnhöfe.

Das können wir bestätigen! The DB S&S hat laut den Daten `5.413` Bahnhöfe. Der Rest, `277`, werden von der DB Regio-Netze unterhalten.

In [27]:
plot_counts('owner')

Betrachtet man die Organisationsbereiche, gibt es viele Stationen in der mitte/süden/westen von Deutschland. Der Norden und Osten liegen hingegen auf den letzten Plätzen.

Es gibt auch einige kleinere Organisationseinheiten für spezielle Regionen.

In [53]:
plot_counts('organisationalUnit')

Tatsächlich sehen wir, dass die führenden Bundesländer Bayern, Baden-Württemberg und Nordrhein-Westfalen (NRW) sind.<br>
Es besteht eine deutliche Lücke zwischen ihnen und dem viertplatzierten Bundesland Hessen. Natürlich müssen wir auch die Größe der Bundesländer berücksichtigen.

Wir können prüfen, welches Bundesland [laut seiner Größe](https://www.statistikportal.de/de/bevoelkerung/flaeche-und-bevoelkerung) die meisten Bahnhöfe hat.
Dazu rufen wir die Größe der Bundesländer ab und setzen sie ins Verhältnis mit der Anzahl der Stationen.

In [54]:
df_states = pd.read_csv('states_size.csv', sep=';')
df_states['size'] = df_states['size'].astype(float)
df_states

,state,size
0,Baden-Württemberg,35747.82
1,Bayern,70541.57
2,Berlin,891.12
3,Brandenburg,29654.35
4,Bremen,419.62
5,Hamburg,755.09
6,Hessen,21115.64
7,Mecklenburg-Vorpommern,23295.45
8,Niedersachsen,47709.82
9,Nordrhein-Westfalen,34112.44


In [56]:
df_grp_states = pd.DataFrame(df_stations.groupby(by='state').count()['id'].sort_values(ascending=False))
df_grp_states.rename(columns={'id': 'count'}, inplace=True)
df_grp_states

,count
state,
Bayern,1025
Baden-Württemberg,720
Nordrhein-Westfalen,711
Hessen,479
Sachsen,478
Rheinland-Pfalz,419
Niedersachsen,357
Brandenburg,310
Sachsen-Anhalt,289


In [57]:
df_germany = pd.DataFrame(index=['count'], data={
    'Deutschland':df_grp_states.sum().values[0]
}).T

df_grp_states = pd.concat([df_grp_states, df_germany])
df_grp_states['state'] = df_grp_states.index
df_grp_states = pd.merge(df_states, df_grp_states, how='left', on='state')

In [58]:
df_grp_states

,state,size,count
0,Baden-Württemberg,35747.82,720
1,Bayern,70541.57,1025
2,Berlin,891.12,133
3,Brandenburg,29654.35,310
4,Bremen,419.62,16
5,Hamburg,755.09,58
6,Hessen,21115.64,479
7,Mecklenburg-Vorpommern,23295.45,180
8,Niedersachsen,47709.82,357
9,Nordrhein-Westfalen,34112.44,711


- Gemäß seiner Größe hat Berlin die meisten Haltestellen, wobei jede Station durchschnittlich etwa 6,7 km² abdeckt. Für eine Hauptstadt ergibt dies Sinn.
- Im Allgemeinen haben alle großen Städte (Hamburg, Bremen) dieses Verhältnis.
- Zuvor waren Bayern und Baden-Württemberg die Regionen mit den meisten Stationen, nun befinden sie sich auf den Plätzen 11 und 8.
- Das Saarland, das die wenigsten Bahnhöfe hat, weist das beste Verhältnis der Fläche zu Stationen unter den Bundesländern auf.
- In Niedersachsen muss eine Station im Durchschnitt etwa 133,6 km² abdecken, das ist viel, wenn man so weit fahren muss, um zur nächsten Station zu gelangen.
- Der Durchschnitt für ganz Deutschland liegt bei 62 km².

In [62]:
df_grp_states['ratio'] = df_grp_states['size']/df_grp_states['count']
df_grp_states.sort_values('ratio', ascending=True).reset_index().drop(columns=['index'])

,state,size,count,ratio
0,Berlin,891.12,133,6.700150
1,Hamburg,755.09,58,13.018793
2,Bremen,419.62,16,26.226250
3,Saarland,2571.11,77,33.391039
4,Sachsen,18449.93,478,38.598180
5,Hessen,21115.64,479,44.082756
6,Rheinland-Pfalz,19858.00,419,47.393795
7,Nordrhein-Westfalen,34112.44,711,47.978115
8,Baden-Württemberg,35747.82,720,49.649750
9,Thüringen,16202.39,289,56.063633


Um das ganze auf einer Karte anzuzeigen, fehlen uns noch die Geo-Koordinaten der Bundesländer. Diese werden wie zuvor abgerufen.

In [60]:
geolocator = Nominatim(user_agent="my_app")

result={}
for entry in df_grp_states['state']:
        result[entry] = geolocator.geocode(entry)

In [63]:
res_dict={}
# extract lat/lon
for index, entry in result.items():
    if entry:
        res_dict[index] = { 'latitude': entry[1][0], 'longitude': entry[1][1] }

geocode_result = pd.DataFrame().from_dict(res_dict).T
geocode_result['state'] = geocode_result.index
geocode_result

,latitude,longitude,state
Baden-Württemberg,48.537750,9.041169,Baden-Württemberg
Bayern,48.946756,11.403872,Bayern
Berlin,52.517037,13.388860,Berlin
Brandenburg,52.845549,13.246130,Brandenburg
Bremen,53.075820,8.807165,Bremen
Hamburg,53.550341,10.000654,Hamburg
Hessen,50.608065,9.028465,Hessen
Mecklenburg-Vorpommern,53.773506,12.575547,Mecklenburg-Vorpommern
Niedersachsen,52.839853,9.075962,Niedersachsen
Nordrhein-Westfalen,51.478921,7.554375,Nordrhein-Westfalen


In [64]:
df_grp_states_geo = pd.merge(df_grp_states, geocode_result, how='left', on='state')
df_grp_states_geo.drop(16, inplace=True) # drop germany

In diesem Fall nutzen wir `plotly`, um die Karte anzuzeigen.

Die größe der Punkte zeigt die Anzahl der Haltestationen an.<br>
Die Farbe gibt das Verhältnis zur Fläche an. Grün steht für deine hohe Dichte an Stationen, rot für eine niedrige.

Hier sehen wir nochmal, dass die *Stadtstaaten* eine hohe Dichte an Haltestationen aufweisen, allerdings absolut gesehen wenige Stationen haben (kleiner Kreis).<br>
Die großen Bundesländer sind eher im mittleren Farbschema, wobei die im Norden rot gefärbt und damit Schlusslicht sind.

In [66]:
fig = px.scatter_geo(df_grp_states_geo,
                     lat='latitude', 
                     lon='longitude',
                     hover_name='state',     # Data to display when hovering over each data point
                     size='count',     # Size of the markers
                     color='ratio',    # Color of the markers
                     color_continuous_scale=['green','orange','red'],
                     projection='mercator',
                     scope='europe',
                     width=650,
                     height=800)  # Map projection

fig.update_geos(center=dict(lon=10, lat=51), projection_scale=10)
fig.show()

#### Verkehrsmittel und Verkehrsverbünde

Da die Verkehrsmittel und Verkehrsverbünde in geschachtelten Listen vorliegen, müssen diese zunächst geebnet werden, um die absolute Anzahl herauszufinden.

In [67]:
transports = []
for entry in df['transportAssociations']:
    try:
        for e in entry:
            transports.append(e)
    except:
        pass

transportAssociations = pd.Series(transports).value_counts()

In [68]:
transports = []
for entry in df['availableTransports']:
    try:
        for e in entry:
            transports.append(e)
    except:
        pass

availableTransports = pd.Series(transports).value_counts()

... Text

In [69]:
px.bar(transportAssociations)

... Text

In [70]:
px.bar(availableTransports)

## Dashboard für Haltestationen

Wir können uns eine Karte anzeigen lassen, die alle Haltestationen mit Bild anzeigt.

Ganz Deutschland führt allerding zu Performanceproblemen, daher hier nur ein Ausschnitt von Baden-Württemberg.

In [104]:
df.dropna(subset = ['latitude'], inplace=True)

In [106]:
df_bw = df[df['state']=='Baden-Württemberg']

Auf dieser Karte kann man sehr schön erkennen, wo die Bahnlienien verlaufen und das es gewisse Regionen gibt, die nicht an die Bahn angeschlossen sind.

Beim Klicken durch die Bilder fällt auch auf, dass die Bahnhöfe häufig ältere Gebäude sind, von den allerdings viele Renoviert wurden.

In [115]:
# Define df for the map. Germany is too much to render.
map_df = df_bw

m = folium.Map(location=[50.111, 8.682],zoom_start=6)
for i in map_df.index:
    html=f"""
    <img src="{map_df['image'][i]}" width="500px">
    <br/>
    <b><p>{map_df['id'][i]}: {map_df['name'][i]}</b></p>
    <p>Transports: {map_df['availableTransports'][i]}</p>
    <p>Associations: {map_df['transportAssociations'][i]}</p>
    """

    parsedHtml = folium.Html(html, script=True)
    popup = folium.Popup(parsedHtml, max_width=2650)

    folium.Marker(location=[ map_df['latitude'][i], map_df['longitude'][i] ], fill_color='#43d9de', radius=8,tooltip=map_df['name'][i], popup=popup).add_to(m)

m